This notebook is a work in progress. I will clean up code, add o

# Multiclass Synthetic Data Experimentation

All testing done in this notebook is for proof of concept.
<br>
The following sections experiment with synthetic generation for multiclass classification. Furthermore, since Linear Discriminant Analysis runs much faster than Random Forest, the SMOTE classifiers are tested on LD instead of RF classifiers

In [83]:
import numpy as np
import pandas as pd
import imblearn
import sklearn
import smote_variants
import os.path
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import smote_variants as sv
from sklearn.model_selection import train_test_split
from sklearn import metrics

print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"imblearn version {imblearn.__version__}")
print(f"sklearn version {sklearn.__version__}")
print(f"smote_variants version {smote_variants.__version__}")



numpy version 1.18.1
pandas version 1.0.1
imblearn version 0.8.1
sklearn version 0.24.1
smote_variants version 0.4.0


In [103]:
# Read csv files into a dataframe
df = pd.read_csv("../../../Data/cleaned/Darknet_phase_1.csv")

traffic_application_to_num = {
    'audio-streaming': 0,
    'browsing': 1,
    'chat': 2,
    'email': 3,
    'file-transfer': 4,
    'p2p': 5,
    'video-streaming': 6,
    'voip': 7
}

traffic_type_to_num = {
    'Non-Tor': 0,
    'NonVPN': 1,
    'Tor': 2,
    'VPN': 3
}

df['Label1'] = df['Label1'].map(traffic_application_to_num)
df['Label'] = df['Label'].map(traffic_type_to_num)
df['Label1'] = df['Label1'].astype(int)
df['Label1'] = df['Label'].astype(int)

In [104]:
X = df.loc[:, df.columns != 'Label1']
y = df['Label1']

# Baseline Linear Discriminant Analysis

In [73]:
ld_classifier = LinearDiscriminantAnalysis()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
ld_classifier.fit(X_train, y_train)

y_pred = ld_classifier.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.6370993391525603
F1: 0.6081881128419068


# Testing a single SMOTE variant with Linear Discriminant Analysis

In [105]:
# Convert the data and target datasets to numpy arrays
X = X.to_numpy()
y = y.to_numpy()

In [106]:
# The smote_variants package wants specific labels for the data
dataset= {'data': X, 'target': y, 'name': 'darknet_multiclass'}

In [107]:
# This can be stored anywhere; however, it takes a lot of space. We will need to change the location later
cache_path= "/mnt/sda1"

if not os.path.exists(cache_path):
    os.makedirs(cache_path)

In [108]:
oversampler= sv.MulticlassOversampling(sv.distance_SMOTE())

X_samp, y_samp= oversampler.sample(X, y)

2022-02-19 13:32:01,415:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-02-19 13:32:01,450:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-19 13:32:01,467:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-19 13:32:11,492:INFO:MulticlassOversampling: Sampling minority class with label: 3
2022-02-19 13:32:11,519:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.42999651799949906, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-19 13:32:20,380:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-19 13:32:20,411:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.3300016155958856, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


In [109]:
X_samp_df = pd.DataFrame(X_samp, columns = ['Protocol', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label'])
y_samp_df = pd.DataFrame(y_samp, columns = ['Label1'])

In [110]:
y_samp_df

,Label1
0,0
1,0
2,0
3,0
4,0
...,...
373230,2
373231,2
373232,2
373233,2


In [111]:
ld_classifier = LinearDiscriminantAnalysis()

X_train, X_test, y_train, y_test = train_test_split(X_samp_df, y_samp_df, test_size=0.2)
ld_classifier.fit(X_train, y_train)

y_pred = ld_classifier.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))

/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.7750746848500275
F1: 0.7741058083293085


In [112]:
traffic_application_to_val = {
    0:'audio-streaming',
    1:'browsing',
    2:'chat',
    3:'email',
    4:'file-transfer',
    5:'p2p',
    6:'video-streaming',
    7:'voip'
}

traffic_type_to_val = {
    0:'Non-Tor',
    1:'NonVPN',
    2:'Tor',
    3:'VPN'
}

X_samp_df['Label'] = X_samp_df['Label'].map(traffic_application_to_val)
y_samp_df['Label1'] = y_samp_df['Label1'].map(traffic_type_to_val)

In [117]:
new_dataset = X_samp_df.join(y_samp_df)

In [120]:
new_dataset.to_csv("../../../Data/synthetic/SMOTE/smote_multiclass.csv")